## Preambule

In [1]:
import warnings

import numpy as np
from tqdm import tqdm

warnings.simplefilter(action="ignore", category=FutureWarning)

%load_ext autoreload
%autoreload all

## Run classes

Data reader class

In [47]:
def get_unfccc_countries(input_path):
    """Load country name and iso as lookup table."""

    columns = {"Name": "name", "Country ISO Code": "iso"}
    return (
        pd.read_excel(
            input_path,
            sheet_name="Country groups",
            usecols=columns.keys(),
        )
        .rename(columns=columns)
        .set_index("name")["iso"]
        .to_dict()
    )

additional_eu_and_earth = {"European Union": "EU", "Earth": "EARTH"}
additional_regions_ssps = {
    "Aruba": "ABW",
    "Bahamas": "BHS",
    "C?te d'Ivoire": "CIV",
    "Cabo Verde": "CPV",
    "Cura?ao": "CUW",
    "Czechia": "CZE",
    "Democratic Republic of the Congo": "COD",
    "French Guiana": "GUF",
    "French Polynesia": "PYF",
    "Gambia": "GMB",
    "Guadeloupe": "GLP",
    "Guam": "GUM",
    "Hong Kong": "HKG",
    "Iran": "IRN",
    "Macao": "MAC",
    "Martinique": "MTQ",
    "Mayotte": "MYT",
    "Moldova": "MDA",
    "New Caledonia": "NCL",
    "Palestine": "PSE",
    "Puerto Rico": "PRI",
    "R?union": "REU",
    "Syria": "SYR",
    "Taiwan": "TWN",
    "Tanzania": "TZA",
    "Turkey": "TUR",
    "United States Virgin Islands": "VIR",
    "United States": "USA",
    "Venezuela": "VEN",
    "Viet Nam": "VNM",
    "Western Sahara": "ESH",
    "World": "EARTH",
}
additional_regions_hdi = {
    "Bahamas": "BHS",
    "Bolivia (Plurinational State of)": "BOL",
    "Cabo Verde": "CPV",
    "China": "TWN",
    "Congo (Democratic Republic of the)": "COD",
    "Congo": "COG",
    "Czechia": "CZE",
    "Egypt": "EGY",
    "Eswatini (Kingdom of)": "SWZ",
    "Gambia": "GMB",
    "Hong Kong, China (SAR)": "HKG",
    "Iran (Islamic Republic of)": "IRN",
    "Korea (Democratic People's Rep. of)": "PRK",
    "Korea (Republic of)": "KOR",
    "Kyrgyzstan": "KGZ",
    "Lao People's Democratic Republic": "LAO",
    "Micronesia (Federated States of)": "FSM",
    "Moldova (Republic of)": "MDA",
    "Palestine, State of": "PSE",
    "Saint Kitts and Nevis": "KNA",
    "Saint Lucia": "LCA",
    "Saint Vincent and the Grenadines": "VCT",
    "Sao Tome and Principe": "STP",
    "Slovakia": "SVK",
    "Tanzania (United Republic of)": "TZA",
    "Türkiye": "TUR",
    "United States": "USA",
    "Venezuela (Bolivarian Republic of)": "VEN",
    "Viet Nam": "VNM",
    "Yemen": "YEM",
}

# How to use
filename = '../data/input/UNFCCC_Parties_Groups_noeu.xlsx'
countries = get_unfccc_countries(filename)
regions = {**countries, **additional_eu_and_earth}
ssp_regions = {**countries, **additional_eu_and_earth, **additional_regions_ssps}
hdi_countries = {**countries, **additional_regions_hdi}

# Get individual countries' ISO
countries.get('Netherlands')  # "NLD"
ssp_regions.get('World')  # "EARTH"
hdi_countries.get('Yemen')  # "YEM"

# Get ISO for a list of countries
list(map(countries.get, ['Netherlands', 'Germany']))
# ['NLD', 'DEU']

# Get ISO for a list of countries with missing data
list(map(additional_regions_ssps.get, ['French Guiana', 'French Polynesia', 'French Fries']))
# ['GUF', 'PYF', None]

# Get ISO for a list of countries with custom default when missing (do we really need this?)
list(map(lambda name: countries.get(name, 'oeps'), ['Ireland', 'South Africa', 'Mordor']))
# ['IRL', 'ZAF', 'oeps']

# The same thing achieved with a convenience function
def get_iso(names, lookup_table, default='oeps'):
    return list(map(lambda name: lookup_table.get(name, default), names))

get_iso(['Belgium', 'France', 'Atlantis'], hdi_countries, default='sunken')
# ['BEL', 'FRA', 'sunken']


['BEL', 'FRA', 'sunken']

In [ ]:
filename = '../input/data/external/HDR21-22_Statistical_Annex_HDI_Table.xlsx'
df = pd.read_excel(filename,sheet_name="Rawdata")

In [9]:
# Testing alternative implementation of read_ssps

from effortsharing.class_datareading import datareading
import xarray as xr
import numpy as np

datareader = datareading("../input.yml")
countries, regions = datareader.read_unfccc_countries()
ssps_old = datareader.read_ssps(regions)
ssps_new = datareader.read_ssps_refactor(regions)

# This fails:
# xr.testing.assert_equal(ssps_new, ssps_old)

# Get the elements that are not the same. 
mismatch1 = ssps_new.GDP.where(ssps_old.GDP != ssps_new.GDP)

# Looks like old version has more NaNs; this yields the same mismatch as above
old_is_nan = np.isnan(ssps_old.GDP)
new_is_not_nan = ~np.isnan(ssps_new.GDP)
mismatch2 = ssps_new.GDP.where((old_is_nan) & (new_is_not_nan))
xr.testing.assert_equal(mismatch1, mismatch2)  # passes, so these are indeed the same

# Inspect which entries are missing in old but present in new version
mismatch_coords = mismatch1.stack(coords=mismatch1.dims).dropna("coords")
np.unique(mismatch_coords.Time)  # Only historical
np.unique(mismatch_coords.Region) # Almost all (186 / 201) regions
np.unique(mismatch_coords.Scenario)  # All scenarios
list(zip(*[mismatch_coords[dim].values for dim in mismatch1.dims]))
# These entries do seem to be present in excel sheet

# ==================================== #
# DATAREADING class                    #
# startyear:  2021
# ==================================== #
- Reading unfccc country data
- Reading GDP and population data from SSPs
- Reading GDP and population data from SSPs


[('SSP1', 'ABW', 1990),
 ('SSP1', 'ABW', 1995),
 ('SSP1', 'ABW', 2000),
 ('SSP1', 'ABW', 2005),
 ('SSP1', 'ABW', 2010),
 ('SSP1', 'ABW', 2015),
 ('SSP1', 'AGO', 1990),
 ('SSP1', 'AGO', 1995),
 ('SSP1', 'AGO', 2000),
 ('SSP1', 'AGO', 2005),
 ('SSP1', 'AGO', 2010),
 ('SSP1', 'AGO', 2015),
 ('SSP1', 'ALB', 1980),
 ('SSP1', 'ALB', 1985),
 ('SSP1', 'ALB', 1990),
 ('SSP1', 'ALB', 1995),
 ('SSP1', 'ALB', 2000),
 ('SSP1', 'ALB', 2005),
 ('SSP1', 'ALB', 2010),
 ('SSP1', 'ALB', 2015),
 ('SSP1', 'ARE', 1980),
 ('SSP1', 'ARE', 1985),
 ('SSP1', 'ARE', 1990),
 ('SSP1', 'ARE', 1995),
 ('SSP1', 'ARE', 2000),
 ('SSP1', 'ARE', 2005),
 ('SSP1', 'ARE', 2010),
 ('SSP1', 'ARE', 2015),
 ('SSP1', 'ARG', 1980),
 ('SSP1', 'ARG', 1985),
 ('SSP1', 'ARG', 1990),
 ('SSP1', 'ARG', 1995),
 ('SSP1', 'ARG', 2000),
 ('SSP1', 'ARG', 2005),
 ('SSP1', 'ARG', 2010),
 ('SSP1', 'ARG', 2015),
 ('SSP1', 'ARM', 1990),
 ('SSP1', 'ARM', 1995),
 ('SSP1', 'ARM', 2000),
 ('SSP1', 'ARM', 2005),
 ('SSP1', 'ARM', 2010),
 ('SSP1', 'ARM',

In [48]:
from effortsharing.class_datareading import datareading

datareader = datareading("../input.yml")
countries, regions = datareader.read_unfccc_countries()
ssp_gdp_population = datareader.read_ssps(regions)
un_population = datareader.read_un_population(countries)
datareader.read_hdi()
# datareader.read_historicalemis_jones()
# datareader.read_ar6()
# datareader.relation_budget_nonco2()
# datareader.determine_global_nonco2_trajectories()
# datareader.determine_global_budgets()
# datareader.determine_global_co2_trajectories()
# datareader.read_baseline()
# datareader.read_ndc()
# datareader.read_ndc_climateresource()
# datareader.merge_xr()
# datareader.add_country_groups()
# datareader.save()
# datareader.country_specific_datareaders()

# ==================================== #
# DATAREADING class                    #
# startyear:  2021
# ==================================== #
- Reading unfccc country data
- Reading GDP and population data from SSPs
- Reading UN population data and gapminder, processed by OWID (for past population)
- Read Human Development Index data


/home/peter/pbl/effort-sharing/src/effortsharing/class_datareading.py:393: RuntimeWarning: invalid value encountered in cast
  "SCG",


AttributeError: 'datareading' object has no attribute 'countries_iso'

In [ ]:
from effortsharing.class_allocation import allocation

allocator = allocation("USA", lulucf="incl", gas="GHG")
allocator.gf()
allocator.pc()
allocator.pcc()
allocator.pcb()
allocator.ecpc()
allocator.ap()
allocator.gdr()
allocator.save()

Allocations specifically for Norway and Netherlands if you want to harmonize to national data

In [ ]:
from effortsharing.class_allocation import allocation

for lulucf in ["incl", "excl"]:
    for gas in ["GHG", "CO2"]:
        for dataread_file in [
            "xr_dataread.nc",
            "xr_dataread_nor.nc",
        ]:  # This is only necessary if for a country a specific historical emissions profile is required beyond what is in Jones (e.g. Norway was requested)
            allocator = allocation(
                "NOR", lulucf=lulucf, gas=gas, dataread_file=dataread_file
            )
            allocator.gf()
            allocator.pc()
            allocator.pcc()
            allocator.pcb()
            allocator.ecpc()
            allocator.ap()
            allocator.gdr()
            allocator.save()
            print("NOR", lulucf, gas, dataread_file, "done")

NOR incl GHG xr_dataread.nc done
NOR incl GHG xr_dataread_nor.nc done
NOR incl CO2 xr_dataread.nc done
NOR incl CO2 xr_dataread_nor.nc done
NOR excl GHG xr_dataread.nc done
NOR excl GHG xr_dataread_nor.nc done
NOR excl CO2 xr_dataread.nc done
NOR excl CO2 xr_dataread_nor.nc done


Allocation class (run all 207 countries)

In [ ]:
from effortsharing.class_allocation import allocation

regions_iso = np.load(
    "K:/Data/Data_effortsharing/DataUpdate_ongoing/all_regions.npy", allow_pickle=True
)
for cty in tqdm(regions_iso):
    allocator = allocation(cty, lulucf="incl", gas="GHG")
    allocator.gf()
    allocator.pc()
    allocator.pcc()
    allocator.pcb()
    allocator.ecpc()
    allocator.ap()
    allocator.gdr()
    allocator.save()

Allocation rules combining approaches (Robiou paper)

In [ ]:
from effortsharing.class_allocation_combinedapproaches import allocation_comb

allocator = allocation_comb()
allocator.ecpc()
allocator.pc()
allocator.discounting_historical_emissions()
allocator.approach1gdp()
allocator.approach1hdi()
allocator.approach2()
allocator.approach2_transition()
allocator.combine()
# allocator.get_relation_2030emis_temp()
allocator.determine_tempoutcomes_discrete()
allocator.save()

Temperature NDC-alignment metric

In [ ]:
from effortsharing.class_tempalign import tempaligning

tempaligner = tempaligning()  # FIRST RUN AGGREGATOR FOR THIS!! (2030 alloc)
tempaligner.get_relation_2030emis_temp()
tempaligner.determine_tempoutcomes()
tempaligner.save()

# ==================================== #
# Initializing tempaligning class        #
# ==================================== #
- Determine relation between 2030-emissions and temperature outcome
- Determine temperature metric


100%|██████████| 6/6 [00:11<00:00,  1.96s/it]


- Save


Reading policy scenarios from ENGAGE

In [ ]:
from effortsharing.class_policyscens import policyscenadding

policyscenner = policyscenadding()
policyscenner.read_engage_data()
policyscenner.filter_and_convert()
policyscenner.add_to_xr()

# ==================================== #
# Initializing policyscenadding class  #
# ==================================== #
- Read ENGAGE scenarios and change region namings
- Filter correct scenarios and convert to xarray object
- Add to overall xrobject


Variance decomposition

In [ ]:
from effortsharing.class_variancedecomp import vardecomposing

vardecomposer = vardecomposing(startyear=2021, gas="GHG", lulucf="incl")
vardecomposer.sobolindices = {}
print("- Starting sobols for fixed years, over many countries")
timeseries = np.arange(2025, 2101, 1)
for year in timeseries:
    print("  Starting with", year)
    xr_cty, ar_time, array_dims, array_inputs, problem, samples = (
        vardecomposer.prepare_global_sobol(year)
    )
    vardecomposer.sobolindices[year] = vardecomposer.apply_decomposition(
        xr_cty, ar_time, array_dims, array_inputs, problem, samples
    )
vardecomposer.save(array_dims, timeseries)

# ==================================== #
# Initializing vardecomposing class    #
# ==================================== #
- Starting sobols for fixed years, over many countries
  Starting with 2025


207it [03:02,  1.13it/s]


  Starting with 2026


207it [02:56,  1.17it/s]


  Starting with 2027


...

207it [02:28,  1.39it/s]


  Starting with 2100


207it [02:27,  1.41it/s]


- Save global results


PermissionError: [Errno 13] Permission denied: 'K:\\Data\\Data_effortsharing\\DataUpdate_ongoing\\startyear_2021\\xr_sobol_GHG_incl.nc'

In [9]:
vardecomposer.save(array_dims, timeseries)

- Save global results
